In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE142610"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE142610"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE142610.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE142610.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE142610.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrative genomic meta-analysis reveals novel molecular insights into cystic fibrosis and ΔF508-CFTR rescue"
!Series_summary	"Cystic fibrosis (CF), caused by mutations to CFTR, leads to severe and progressive lung disease. The most common mutant, ΔF508-CFTR, undergoes proteasomal degradation, depleting its anion channel function.  “Proteostasis” pathways, i.e. those relevant to protein processing and trafficking, are altered in cells with ΔF508-CFTR and can be modulated to partially rescue protein function. However, many details regarding proteostasis modulation, and its relevance to CF and ΔF508-CFTR rescue, remain poorly understood. To shed light on this, we re-analyzed public datasets characterizing transcription in CF vs. non-CF epithelia from human and pig airways, and also profiled established temperature, genetic, and chemical interventions that rescue ΔF508-CFTR. Meta-analysis yielded a core disease signature and two core rescue signatur

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
import re
from typing import Callable, Optional, Dict, Any, Union
import json

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a gene expression dataset 
# analyzing transcriptional changes in CF, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Based on the characteristics dictionary, we see "treatment" at index 1
# which can be used to determine CF vs non-CF status
trait_row = 1  # The row with treatment information

# There is no age data available in the sample characteristics
age_row = None

# There is no gender data available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert the treatment value to a binary trait (CF status).
    1 = CF-like condition (DMSO, scrambled, no treatment as controls)
    0 = Rescue interventions (treatments aimed at rescuing ΔF508-CFTR)
    
    Args:
        value: The treatment value from the dataset
    
    Returns:
        1 for CF-like (control) conditions, 0 for rescue interventions, None for unknown
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Control conditions (CF-like)
    if any(x in value.lower() for x in ['dmso', 'scrambled dsirna', 'no treatment']):
        return 1
    # Rescue interventions
    elif any(x in value.lower() for x in ['temperature: 27', 'sin3a', 'syvn1', 'nedd8', 'mir-138', 'c18', 'corrector']):
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Placeholder function since age data is not available."""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Placeholder function since gender data is not available."""
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only if trait data is available
if trait_row is not None:
    try:
        # Create a DataFrame in the expected format for geo_select_clinical_features
        # Samples as columns, features as rows
        sample_characteristics = {
            0: ['tag: Cell line: CFBE'],
            1: ['treatment: DMSO for 24h', 
                'temperature: 40°C incubation for 24h followed by 27°C incubation for 24h', 
                'treatment: NEDD8 DsiRNA + 6µM Corrector Compound C18 treatment for 24h', 
                'treatment: Scrambled DsiRNA', 
                'temperature: 27°C incubation for 24h', 
                'treatment: SIN3A DsiRNA', 
                'temperature: 37°C incubation for 24h', 
                'treatment: SYVN1 DsiRNA', 
                'treatment: 6µM Corrector Compound C18 treatment for 24h', 
                'treatment: No treatment', 
                'treatment: miR-138 mimic', 
                'treatment: SYVN1 DsiRNA + 6µM Corrector Compound C18 treatment for 24h', 
                'temperature: 40°C incubation for 24h', 
                'treatment: NEDD8 DsiRNA']
        }
        
        # Create a list of sample IDs
        unique_treatments = sample_characteristics[1]
        sample_ids = [f"Sample_{i+1}" for i in range(len(unique_treatments))]
        
        # Create the clinical data DataFrame with samples as columns
        clinical_data = pd.DataFrame(index=range(max(sample_characteristics.keys())+1))
        
        for i, sample_id in enumerate(sample_ids):
            clinical_data[sample_id] = None
            
            # Assign cell line info to all samples
            clinical_data.at[0, sample_id] = sample_characteristics[0][0]
            
            # Assign treatment info
            clinical_data.at[1, sample_id] = unique_treatments[i]
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Clinical Data Preview:", preview)
        
        # Save the processed clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {str(e)}")
        print("Clinical data processing skipped.")


Clinical Data Preview: {'Sample_1': [1.0], 'Sample_2': [nan], 'Sample_3': [0.0], 'Sample_4': [1.0], 'Sample_5': [nan], 'Sample_6': [0.0], 'Sample_7': [nan], 'Sample_8': [0.0], 'Sample_9': [0.0], 'Sample_10': [1.0], 'Sample_11': [0.0], 'Sample_12': [0.0], 'Sample_13': [nan], 'Sample_14': [0.0]}
Clinical data saved to ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE142610.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 66
Header line: "ID_REF"	"GSM4232834"	"GSM4232835"	"GSM4232836"	"GSM4232837"	"GSM4232838"	"GSM4232839"	"GSM4232840"	"GSM4232841"	"GSM4232842"	"GSM4232843"	"GSM4232844"	"GSM4232845"	"GSM4232846"	"GSM4232847"	"GSM4232848"	"GSM4232849"	"GSM4232850"	"GSM4232851"	"GSM4232852"	"GSM4232853"	"GSM4232854"	"GSM4232855"	"GSM4232856"	"GSM4232857"	"GSM4232858"	"GSM4232859"	"GSM4232860"	"GSM4232861"	"GSM4232862"	"GSM4232863"	"GSM4232864"	"GSM4232865"	"GSM4232866"	"GSM4232867"	"GSM4232868"	"GSM4232869"	"GSM4232870"	"GSM4232871"	"GSM4232872"	"GSM4232873"	"GSM4232874"	"GSM4232875"	"GSM4232876"	"GSM4232877"	"GSM4232878"	"GSM4232879"	"GSM4232880"	"GSM4232881"	"GSM4232882"	"GSM4232883"	"GSM4232884"	"GSM4232885"	"GSM4232886"	"GSM4232887"	"GSM4232888"	"GSM4232889"	"GSM4232890"	"GSM4232891"	"GSM4232892"	"GSM4232893"
First data line: "7A5"	7.00047	7.4364	7.2259	6.95089	7.01398	6.94179	6.35476	6.39446	7.04405	6.67603	6.38158	6.87048	6.78098	6.94703	7.00125	7.0633	6.01448	7.10264	6.872

### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers
# Looking at the identifiers from the output, we can see entries like:
# '7A5', 'A1BG', 'A1CF', 'A2BP1', etc.

# These appear to be standard human gene symbols. 
# For example:
# - A1BG is Alpha-1-B Glycoprotein
# - A2M is Alpha-2-Macroglobulin
# - AAAS is Achalasia, Adrenocortical Insufficiency, Alacrimia syndrome gene

# While some identifiers might be less common (like 7A5), the majority appear to be
# standard HGNC gene symbols, so no mapping should be required

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Properly extract original clinical data to match GSM IDs
# First, let's get the proper mapping between GSM IDs and treatment conditions
# Extract the relevant lines from the SOFT file for sample information
with gzip.open(soft_file, 'rt') as f:
    sample_info_lines = []
    current_gsm = None
    for line in f:
        line = line.strip()
        if line.startswith("^SAMPLE = "):
            current_gsm = line.split(" = ")[1]
        elif line.startswith("!Sample_title") and current_gsm:
            title = line.split(" = ")[1].strip('"')
            sample_info_lines.append((current_gsm, title))

# Create a mapping of GSM IDs to treatment conditions
gsm_to_treatment = {}
for gsm, title in sample_info_lines:
    gsm_to_treatment[gsm] = title

print(f"Found treatment information for {len(gsm_to_treatment)} samples")

# Create clinical data with real GSM IDs
clinical_data = pd.DataFrame(index=normalized_gene_data.columns)

# Assign trait values based on treatment descriptions
# 1 = CF-like condition (control)
# 0 = Rescue intervention
clinical_data[trait] = clinical_data.index.map(lambda gsm: 1 if any(x in gsm_to_treatment.get(gsm, "").lower() 
                                            for x in ['dmso', 'scrambled', 'control', 'untreated']) 
                                    else 0 if gsm in gsm_to_treatment else None)

print(f"Clinical data created with {len(clinical_data)} samples")
print(clinical_data[trait].value_counts())

# Link the clinical and genetic data
linked_data = clinical_data.join(normalized_gene_data.T)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait is biased
trait_type = 'binary' if len(linked_data[trait].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# 5. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=linked_data,
    note=f"Dataset contains gene expression data comparing rescue interventions with control conditions in CFBE cells."
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20747, 60)
First few genes with their expression values after normalization:
          GSM4232834  GSM4232835  GSM4232836  GSM4232837  GSM4232838  \
ID                                                                     
A1BG         4.79408     4.77433     5.09248     5.12294     5.22396   
A1BG-AS1     4.41521     4.10095     4.19279     4.15799     4.01244   
A1CF         4.47919     4.49296     4.96132     4.61623     4.62902   
A2M          4.18512     3.43994     4.08894     3.50579     3.90165   
A2ML1        4.53153     3.44832     4.08500     2.97268     4.07312   

          GSM4232839  GSM4232840  GSM4232841  GSM4232842  GSM4232843  ...  \
ID                                                                    ...   
A1BG         4.83021     5.07336     4.71037     5.22138     5.04408  ...   
A1BG-AS1     4.37280     4.83188     4.62063     4.36214     3.76720  ...   
A1CF         4.61928     4.64433     4.49737     4.74431     4.53624  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE142610.csv


Found treatment information for 60 samples
Clinical data created with 60 samples
Cystic_Fibrosis
0    52
1     8
Name: count, dtype: int64
Linked data shape: (60, 20748)


Data shape after handling missing values: (60, 20748)
For the feature 'Cystic_Fibrosis', the least common label is '1' with 8 occurrences. This represents 13.33% of the dataset.


Linked data saved to ../../output/preprocess/Cystic_Fibrosis/GSE142610.csv
